In [1]:
import sys, os
sys.argv = [''] #fix parser issue

import tensorflow as tf
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from tqdm import trange

from networks import CNN
from parser import args
from summary import Summary
import utils


2023-02-08 12:41:28.470618: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 12:41:30.028121: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-08 12:41:30.028244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-08 12:41:30.028258: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [9]:
args.eval_during_training = True

In [3]:
def eval_model(network, criterion, test_loader, summary = None):
    accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='Accuracy')
    labels = []
    test_iter = iter(test_loader)
#     for eval_steps in trange(len(test_loader), leave=False, file=sys.stdout,
#                             desc='   Evaluation Progress', unit_scale=False):
    for eval_steps in range(len(test_loader)):
        input, target = next(test_iter)

        if(len(input.shape) <= 3):
            input = tf.expand_dims(input, -1)

        pred = network(input)
        labels += [tf.argmax(pred, axis=1)]
        loss = criterion(target, pred)
        accuracy(target, pred)
        if(summary is not None):
            summary.step(loss, target, pred, eval=True)
    return accuracy.result() * 100, np.array(labels)

In [4]:
def training_step(network, criterion, data, summary, optimizer):
    input, target = data
    loss = 0
    with tf.GradientTape() as tape:
        pred = network(input)
        loss = criterion(target, pred)

    gradients = tape.gradient(loss, network.trainable_variables,
                                    unconnected_gradients=tf.UnconnectedGradients.ZERO)
    optimizer.apply_gradients(zip(gradients, network.trainable_variables))

    summary.step(loss, target, pred, eval=False)

    del tape


In [5]:
def train(network, optimizer, criterion, checkpoint, checkpoint_manager,
            train_loader, test_loader, summary):

    training_steps = len(train_loader)
    steps_done = checkpoint.step.numpy() + 1
    train_iter = iter(train_loader)
    x, y = tf.random.uniform([args.batch_size,28,28,1]), tf.random.uniform([args.batch_size,1])
    t_step = tf.function(training_step).get_concrete_function(network, criterion, (x,y),summary, optimizer)
        
    for steps in trange(int(steps_done), training_steps, desc='Training Progress', file=sys.stdout):
        # fetch data
        input, target = next(train_iter)

        # expand input shape in case of only 1 channel
        if(len(input.shape) <= 3):
            input = tf.expand_dims(input, -1)
        t_step(network, criterion,(input, target),summary)

        # evaluation step
        if(args.eval_during_training and steps > 0
                    and steps % args.num_eval_steps == 0):
            eval_model(network, criterion, test_loader, summary)
            summary.write(steps, eval=True)

        #  log training accuracy and update checkpoint
        if(steps % args.num_summary_steps == 0):
            summary.write(steps, eval=False)
            if(summary.max_test_accuracy <= checkpoint.best_test):
                checkpoint.best_train.assign(summary.max_train_accuracy)
                checkpoint.best_test.assign(summary.max_test_accuracy)
                checkpoint.step.assign(steps)
                checkpoint_manager.save(checkpoint_number=steps//args.num_summary_steps)

    # evaluate model after finishing training epochs
    accuracy, predictions = eval_model(network, criterion, test_loader,summary)

    if(summary.max_test_accuracy <= checkpoint.best_test):
        checkpoint.best_train.assign(summary.max_train_accuracy)
        checkpoint.best_test.assign(summary.max_test_accuracy)
        checkpoint.step.assign(steps)
        checkpoint_manager.save(checkpoint_number=steps//args.num_summary_steps)

    summary.write(training_steps + 1, eval=True)
    summary.write(training_steps + 1, eval=False)

    print('Training accuracy: ', summary.max_train_accuracy)
    print('Evaluation accuracy: ', summary.max_test_accuracy)
    summary.writer.close()
    return accuracy, predictions

In [6]:
# Make sure computing on cuda
print('GPUs found: ', tf.config.list_physical_devices('GPU'))

GPUs found:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-02-08 12:41:31.443663: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 12:41:31.513614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 12:41:31.514609: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [7]:
tf.random.set_seed(args.random_seed)

# Training a 'client' model on MNIST

### Random initialization

In [16]:
experiment = 'c_mnist'
args.dataset = 'mnist'

exp_path = os.path.join(args.model_dir, experiment)
utils.initialize_dirs(exp_path)

summary = Summary(experiment)
network = CNN()
optimizer = utils.initialize_optimizer()
criterion = utils.initialize_criterion()
checkpoint, checkpoint_manager = utils.initialize_checkpoint(network, optimizer, exp_path)

# data loading
train_loader, test_loader, targets = utils.initialize_dataloaders()

#train client network
accuracy, predictions = train(network, optimizer, criterion, checkpoint, checkpoint_manager, train_loader, test_loader, summary)
p, r, f, s = precision_recall_fscore_support(targets, predictions.reshape(-1), average=None,labels=np.arange(20))
    
print('Random Initialization MNIST model:')
print(accuracy.numpy())
print(r)

2023-02-08 11:01:29.761000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 11:01:29.761536: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 11:01:29.762094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 11:01:29.762504: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Training Progress:   0%|                             | 0/131249 [00:00<?, ?it/s]

2023-02-08 11:01:32.517391: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2023-02-08 11:01:32.937848: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-08 11:01:33.888687: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f04a38847b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-08 11:01:33.888717: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-02-08 11:01:33.927751: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-08 11:01:34.205173: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-08 11:01:34.258125: I tensorflow/compiler/jit/xla_compilati

Training Progress: 100%|███████████████| 131249/131249 [06:20<00:00, 345.17it/s]
Training accuracy:  tf.Tensor(100.0, shape=(), dtype=float32)
Evaluation accuracy:  tf.Tensor(98.96, shape=(), dtype=float32)
Random Initialization MNIST model:
98.909996
[0.99693878 0.99647577 0.98643411 0.98811881 0.98879837 0.98766816
 0.9874739  0.98735409 0.9825462  0.98810704 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]


/home/anwar/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anwar/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Set seed for initialization

In [8]:
experiment = 'c_mnist_setseed'
args.dataset = 'mnist'

exp_path = os.path.join(args.model_dir, experiment)
utils.initialize_dirs(exp_path)

summary = Summary(experiment)
network = CNN(42)
optimizer = utils.initialize_optimizer()
criterion = utils.initialize_criterion()
checkpoint, checkpoint_manager = utils.initialize_checkpoint(network, optimizer, exp_path)

# data loading
train_loader, test_loader, targets = utils.initialize_dataloaders()

#train client network
accuracy, predictions =train(network, optimizer, criterion, checkpoint, checkpoint_manager, train_loader, test_loader, summary)
p, r, f, s = precision_recall_fscore_support(targets, predictions.reshape(-1), average=None,labels=np.arange(20))
    
print('Set Seed Initialization MNIST model:')
print(accuracy.numpy())
print(r)

2023-02-08 11:46:23.850052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 11:46:23.850959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 11:46:23.851380: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 11:46:23.851678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Training Progress:   0%|                             | 0/131249 [00:00<?, ?it/s]

2023-02-08 11:46:27.057959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2023-02-08 11:46:28.104811: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-08 11:46:29.687679: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f6fae05c5c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-08 11:46:29.687702: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-02-08 11:46:29.712645: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-08 11:46:29.971559: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-08 11:46:30.022520: I tensorflow/compiler/jit/xla_compilati

Training Progress: 100%|███████████████| 131249/131249 [05:05<00:00, 428.93it/s]
Training accuracy:  tf.Tensor(100.0, shape=(), dtype=float32)
Evaluation accuracy:  tf.Tensor(98.94, shape=(), dtype=float32)
Set Seed Initialization MNIST model:
98.9
[0.99489796 0.99471366 0.98546512 0.98811881 0.99592668 0.98542601
 0.98643006 0.98929961 0.98562628 0.98315164 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]


/home/anwar/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anwar/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Training a 'client' model on Fashion MNIST

### Random initialization

In [8]:
tf.keras.backend.clear_session()
experiment = 'c_fmnist'
args.dataset = 'fmnist'

exp_path = os.path.join(args.model_dir, experiment)
utils.initialize_dirs(exp_path)

# data loading
train_loader, test_loader, targets = utils.initialize_dataloaders()

summary = Summary(experiment)
network = CNN()
network.build([1,28,28,1])
optimizer = utils.initialize_optimizer()
criterion = utils.initialize_criterion()
checkpoint, checkpoint_manager = utils.initialize_checkpoint(network, optimizer, exp_path)


#train client network
accuracy, predictions = train(network, optimizer, criterion, checkpoint, checkpoint_manager, train_loader, test_loader, summary)
p, r, f, s = precision_recall_fscore_support(targets, predictions.reshape(-1), average=None,labels=np.arange(20))
    
print('Random Initialization Fashion MNIST model:')
print(accuracy.numpy())
print(r)

2023-02-08 12:41:41.318441: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 12:41:41.318854: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 12:41:41.319241: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 12:41:41.319526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Training Progress:   0%|                             | 0/131249 [00:00<?, ?it/s]


TypeError: ConcreteFunction training_step(network, criterion, data, summary, optimizer) was constructed with UnknownArgument value <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff0c026e210> in network, but was called with CNN value <networks.s_model.CNN object at 0x7ff0cf8e0810>.

### Set seed for initialization

In [ ]:
experiment = 'c_fmnist_setseed'
args.dataset = 'fmnist'

exp_path = os.path.join(args.model_dir, experiment)
utils.initialize_dirs(exp_path)

summary = Summary(experiment)
network = CNN(42)
optimizer = utils.initialize_optimizer()
criterion = utils.initialize_criterion()
checkpoint, checkpoint_manager = utils.initialize_checkpoint(network, optimizer, exp_path)

# data loading
train_loader, test_loader, targets = utils.initialize_dataloaders()

#train client network
accuracy, predictions = train(network, optimizer, criterion, checkpoint, checkpoint_manager, train_loader, test_loader, summary)
p, r, f, s = precision_recall_fscore_support(targets, predictions.reshape(-1), average=None,labels=np.arange(20))
    
print('Set Seed Initialization MNIST model:')
print(accuracy.numpy())
print(r)

## Create dataset containing both MNIST and FMNIST

In [ ]:
mnist = tf.keras.datasets.mnist
(m_x_train, m_y_train), (m_x_test, m_y_test) = mnist.load_data()
m_x_train = tf.cast(m_x_train, dtype=tf.float32)
m_x_test = tf.cast(m_x_test, dtype=tf.float32)
m_x_train, m_x_test = m_x_train / 255.0, m_x_test / 255.0

fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train_old), (x_test, y_test_old) = fmnist.load_data()
y_train = y_train_old + 10
y_test = y_test_old + 10
x_train = tf.cast(x_train, dtype=tf.float32)
x_test = tf.cast(x_test, dtype=tf.float32)
x_train, x_test = x_train / 255.0, x_test / 255.0

f_x_train = tf.concat([x_train, m_x_train],0)
f_x_test = tf.concat([x_test, m_x_test],0)
f_y_train = tf.concat([y_train, m_y_train],0)
f_y_test = tf.concat([y_test, m_y_test],0)

full_train_loader = tf.data.Dataset.from_tensor_slices((f_x_train, f_y_train)).shuffle(x_train.shape[0]).repeat(args.epochs).batch(args.batch_size)
full_test_loader = tf.data.Dataset.from_tensor_slices((f_x_test, f_y_test)).batch(args.eval_batch_size)

In [ ]:
clients = ['c_mnist', 'c_fmnist']
clients_setseed = ['c_mnist_setseed', 'c_fmnist_setseed']

n_clients = len(clients)

# Creating Federated Model

In [11]:
federal_model = CNN()
federal_model(tf.random.uniform([1,28,28,1])) #initialize network

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[-0.10439795,  0.12120815, -0.01573612,  0.04494804,  0.05239639,
         0.13556068, -0.00365166, -0.07352076, -0.06002626,  0.16004483,
         0.01198496, -0.00214229, -0.01947405,  0.07039872,  0.01709209,
        -0.05730027,  0.07044452, -0.2693583 , -0.03279328,  0.07567452]],
      dtype=float32)>

## Collecting client models

### Randomly initialized models

In [10]:
cmodels=[]
x = tf.random.uniform([1,28,28,1])
for c in clients:
        c_path = os.path.join(args.model_dir, c)
        c_network = CNN()
        c_optimizer = utils.initialize_optimizer()
        c_checkpoint, c_checkpoint_manager = utils.initialize_checkpoint(c_network, c_optimizer, c_path)
        c_network(x) # restore deferred variables
        cmodels += [c_network]

2023-02-08 10:40:54.509994: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8101
2023-02-08 10:40:55.393898: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Resuming training from trained_models/c_fmnist/ckpt-4...


### Averaging models initialized with the same weights

In [14]:
cmodels=[]
x = tf.random.uniform([1,28,28,1])
for c in clients_setseed:
        c_path = os.path.join(args.model_dir, c)
        c_network = CNN()
        c_optimizer = utils.initialize_optimizer()
        c_checkpoint, c_checkpoint_manager = utils.initialize_checkpoint(c_network, c_optimizer, c_path)
        c_network(x) # restore deferred variables
        cmodels += [c_network]

Resuming training from trained_models/c_mnist_setseed/ckpt-656...
Resuming training from trained_models/c_fmnist_setseed/ckpt-656...


## Aggregating the weights of the models

In [12]:
conv1 = tf.zeros_like(federal_model.conv1.kernel)
conv1_b = tf.zeros_like(federal_model.conv1.bias)

conv2 = tf.zeros_like(federal_model.conv2.kernel)
conv2_b = tf.zeros_like(federal_model.conv2.bias)

fc1 = tf.zeros_like(federal_model.fc1.kernel)
fc1_b = tf.zeros_like(federal_model.fc1.bias)
                        
out = tf.zeros_like(federal_model.out.kernel)
out_b = tf.zeros_like(federal_model.out.bias)
for i in range(len(cmodels)):
    conv1 += cmodels[i].conv1.kernel / len(cmodels)
    conv1_b += cmodels[i].conv1.bias / len(cmodels)
    
    conv2 += cmodels[i].conv2.kernel / len(cmodels)
    conv2_b += cmodels[i].conv2.bias / len(cmodels)
    
    fc1 += cmodels[i].fc1.kernel / len(cmodels)
    fc1_b += cmodels[i].fc1.bias / len(cmodels)
    
    out += cmodels[i].out.kernel / len(cmodels)
    out_b += cmodels[i].out.bias / len(cmodels)
    

## Federated model equals the average weight of the client models

In [13]:
federal_model.conv1.kernel = conv1
federal_model.conv1.bias = conv1_b

federal_model.conv2.kernel = conv2
federal_model.conv2.bias = conv2_b

federal_model.fc1.kernel = fc1
federal_model.fc1.bias = fc1_b

federal_model.out.kernel = out
federal_model.out.bias = out_b

In [14]:
criterion = utils.initialize_criterion()

### Random Initialization

In [17]:
# evaluate federated network
accuracy, labels = eval_model(federal_model, criterion, full_test_loader)
p, r, f, s = precision_recall_fscore_support(f_y_test, labels.reshape(-1), average=None,labels=np.arange(20))

print(accuracy.numpy())
print(r)

17.92
[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.883
 0.    0.    0.996 0.723 0.    0.982 0.    0.   ]


/home/anwar/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Set seed Initialization

In [18]:
# evaluate federated network
accuracy, labels = eval_model(federal_model, criterion, test_loader)
p, r, f, s = precision_recall_fscore_support(f_y_test, labels.reshape(-1), average=None,labels=np.arange(20))

print(accuracy.numpy())
print(r)

2023-02-08 10:31:19.600217: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 376320000 exceeds 10% of free system memory.


35.695
[0.36938776 0.04493392 0.21414729 0.44455446 0.39002037 0.14686099
 0.05845511 0.12062257 0.8275154  0.02081269 0.056      0.841
 0.945      0.012      0.08       0.855      0.09       0.565
 0.939      0.152     ]


## Averaging only the last layer

### Random Initialization

In [19]:
for i in range(len(cmodels)):
    federal_model.conv1.kernel = cmodels[i].conv1.kernel
    federal_model.conv1.bias = cmodels[i].conv1.bias

    federal_model.conv2.kernel = cmodels[i].conv2.kernel
    federal_model.conv2.bias = cmodels[i].conv2.bias

    federal_model.fc1.kernel = cmodels[i].fc1.kernel
    federal_model.fc1.bias = cmodels[i].fc1.bias

    federal_model.out.kernel = out
    federal_model.out.bias = out_b

    # evaluate federated network
    accuracy, labels = eval_model(federal_model, criterion, full_test_loader)
    p, r, f, s = precision_recall_fscore_support(f_y_test, labels.reshape(-1), average=None,labels=np.arange(20))
    
    print('Client model', i, ':')
    print('accuracy', accuracy.numpy())
    print(r)

/home/anwar/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Client model 0 :
accuracy 5.7999997
[0.         0.         0.         0.         0.00712831 0.0044843
 0.         0.03015564 0.         0.         0.         0.
 0.013      0.105      0.016      0.007      0.912      0.065
 0.         0.        ]
Client model 1 :
accuracy 17.785
[0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.974
 0.    0.    0.946 0.645 0.    0.992 0.    0.   ]


/home/anwar/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Set seed initialization

In [ ]:
for i in range(len(cmodels)):
    federal_model.conv1.kernel = cmodels[i].conv1.kernel
    federal_model.conv1.bias = cmodels[i].conv1.bias

    federal_model.conv2.kernel = cmodels[i].conv2.kernel
    federal_model.conv2.bias = cmodels[i].conv2.bias

    federal_model.fc1.kernel = cmodels[i].fc1.kernel
    federal_model.fc1.bias = cmodels[i].fc1.bias

    federal_model.out.kernel = out
    federal_model.out.bias = out_b

    # evaluate federated network
    accuracy, labels = eval_model(federal_model, criterion, test_loader)
    p, r, f, s = precision_recall_fscore_support(f_y_test, labels.reshape(-1), average=None,labels=np.arange(20))
    
    print('Client model', i, ':')
    print('accuracy', accuracy.numpy())
    print(r)